In [73]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [74]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:21]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [75]:
country_list[0] = ""

In [76]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '4120',
 '5330',
 '4279',
 '4759',
 '4190',
 '5520',
 '5830',
 '4419',
 '5570',
 '5490',
 '3510',
 '5170',
 '5081',
 '5590']

In [43]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time==from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "data.parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "data.parquet"
    
    
    #if os.path.exists(out_file):
        
     #   print("Already have downloaded file")
        
     #   continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)


<Response [200]>
5700
<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
4120
<Response [200]>
5330
<Response [200]>
4279
<Response [200]>
4759
<Response [200]>
4190
<Response [200]>
5520
<Response [200]>
5830
<Response [200]>
4419
<Response [200]>
5570
<Response [200]>
5490
<Response [200]>
3510
<Response [200]>
5170
<Response [200]>
5081
<Response [200]>
5590
<Response [200]>


In [77]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data.parquet"

    foo = pq.read_table(file).to_pandas()
    
    df = df.append(foo)
    
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)                   

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top20-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [78]:
df

I_COMMODITY  \
CTY_NAME                I_COMMODITY_SDESC        time                     
TOTAL FOR ALL COUNTRIES HS CODE 01, LIVE ANIMALS 2013-01-01          01   
                                                 2013-02-01          01   
                                                 2013-03-01          01   
                                                 2013-04-01          01   
                                                 2013-05-01          01   
...                                                                 ...   
VIETNAM                 ALL PRODUCTS             2020-06-01          00   
                                                 2020-07-01          00   
                                                 2020-08-01          00   
                                                 2020-09-01          00   
                                                 2020-10-01          00   

                                                                  imports  
CTY_NAME                I_COMMODITY_SDESC        time                      
TOTAL FOR ALL COUNTRIES HS CODE 01, LIVE ANIMALS 2013-01-01  2.213131e+08  
                                                 2013-02-01  2.460921e+08  
                                                 2013-03-01  2.612444e+08  
                                                 2013-04-01  2.711339e+08  
                                                 2013-05-01  2.097853e+08  
...                                                                   ...  
VIETNAM                 ALL PRODUCTS             2020-06-01  5.719709e+09  
                                                 2020-07-01  7.098822e+09  
                                                 2020-08-01  7.655598e+09  
                                                 2020-09-01  7.911289e+09  
                                                 2020-10-01  8.192072e+09  

[185352 rows x 2 columns]